In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix, r2_score
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings('ignore')

# Настройки отображения
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("deep")

In [ ]:
# Загрузка данных
df = pd.read_csv('healthcare-dataset-stroke-data.csv')

# Отобразим первые строки и информацию
print("Первые 5 строк:")
display(df.head())

print("\nИнформация о данных:")
df.info()

print("\nПропущенные значения:")
print(df.isnull().sum())

In [ ]:
# Удалим строки с пропущенными значениями в 'bmi' (т.к. анализ по age vs bmi)
df_clean = df.dropna(subset=['bmi']).copy()

# Проверим типы данных
print("Типы данных после очистки:")
df_clean[['age', 'bmi']].dtypes

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clean, x='age', y='bmi', alpha=0.6)
plt.title('Зависимость BMI от возраста')
plt.xlabel('Возраст (age)')
plt.ylabel('Индекс массы тела (bmi)')
plt.show()

In [ ]:
desc_stats = df_clean[['age', 'bmi']].describe()
display(desc_stats)

# Выводы (можно раскомментировать для печати)
print("\nВыводы:")
print("- Возраст варьируется от {:.1f} до {:.1f} лет.".format(desc_stats.loc['min', 'age'], desc_stats.loc['max', 'age']))
print("- BMI варьируется от {:.2f} до {:.2f}.".format(desc_stats.loc['min', 'bmi'], desc_stats.loc['max', 'bmi']))
print("- Средний возраст: {:.1f}, средний BMI: {:.2f}".format(desc_stats.loc['mean', 'age'], desc_stats.loc['mean', 'bmi']))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Гистограммы и Q-Q plot
for i, col in enumerate(['age', 'bmi']):
    axes[i].hist(df_clean[col], bins=30, density=True, alpha=0.7, color='skyblue', edgecolor='black')
    axes[i].set_title(f'Гистограмма {col}')
    
    # Добавим нормальную кривую
    mu, sigma = df_clean[col].mean(), df_clean[col].std()
    x = np.linspace(df_clean[col].min(), df_clean[col].max(), 100)
    axes[i].plot(x, stats.norm.pdf(x, mu, sigma), 'r--', linewidth=2)

plt.tight_layout()
plt.show()

# Тест Шапиро-Уилка
for col in ['age', 'bmi']:
    stat, p = stats.shapiro(df_clean[col].sample(min(5000, len(df_clean))))  # ограничение из-за размера
    print(f"Shapiro-Wilk test для {col}: p-value = {p:.4f} → {'не нормальное' if p < 0.05 else 'нормальное'}")

In [ ]:
# Используем IQR для обнаружения выбросов в 'bmi'
Q1 = df_clean['bmi'].quantile(0.25)
Q3 = df_clean['bmi'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_clean[(df_clean['bmi'] < lower_bound) | (df_clean['bmi'] > upper_bound)]
print(f"Найдено {len(outliers)} выбросов по BMI.")

# Решим не удалять выбросы (медицинские данные)
df_no_outliers = df_clean  # можно заменить на фильтрацию, если нужно

In [ ]:
corr = df_clean[['age', 'bmi']].corr()
print("Матрица корреляции (Пирсона):")
display(corr)

sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Корреляция между age и bmi')
plt.show()

print(f"\nКоэффициент корреляции: r = {corr.iloc[0,1]:.3f}")

In [ ]:
model = smf.ols('bmi ~ age', data=df_clean).fit()
print(model.summary())

In [ ]:
# Подготовка данных
X = df_clean[['age']].values
y = df_clean['bmi'].values

# Создадим полиномиальные признаки (степень 2)
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

# Обучение модели
poly_reg = LinearRegression()
poly_reg.fit(X_poly, y)

# Прогноз
y_poly_pred = poly_reg.predict(X_poly)

# R² для полиномиальной модели
r2_poly = r2_score(y, y_poly_pred)
print(f"R² (полином 2-й степени): {r2_poly:.4f}")

In [ ]:
# Линейная регрессия через sklearn
lin_reg = LinearRegression()
lin_reg.fit(X, y)
y_lin_pred = lin_reg.predict(X)
r2_lin = r2_score(y, y_lin_pred)

print(f"R² (линейная модель):        {r2_lin:.4f}")
print(f"R² (полиномиальная модель):  {r2_poly:.4f}")

if r2_poly > r2_lin:
    print("\nПолиномиальная модель лучше описывает данные.")
else:
    print("\nЛинейная модель предпочтительнее (или разница незначительна).")

In [ ]:
# Сортируем для гладкой линии
sort_idx = np.argsort(X.flatten())
X_sorted = X[sort_idx].flatten()
y_lin_sorted = y_lin_pred[sort_idx]
y_poly_sorted = y_poly_pred[sort_idx]

plt.figure(figsize=(12, 7))
plt.scatter(df_clean['age'], df_clean['bmi'], alpha=0.5, label='Данные', s=10)
plt.plot(X_sorted, y_lin_sorted, color='red', linewidth=2, label='Линейная регрессия')
plt.plot(X_sorted, y_poly_sorted, color='green', linewidth=2, label='Полиномиальная регрессия (2-я степень)')
plt.xlabel('Возраст (age)')
plt.ylabel('BMI')
plt.title('Сравнение линейной и полиномиальной моделей')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Задача 2: avg_glucose_level (количественная) vs stroke (категориальная)
df_task2 = df.dropna(subset=['avg_glucose_level']).copy()
print(f"Данные для задачи 2: {len(df_task2)} записей")

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_task2, x='stroke', y='avg_glucose_level')
plt.title('Уровень глюкозы у пациентов с и без инсульта')
plt.xlabel('Инсульт (0 = нет, 1 = да)')
plt.ylabel('Средний уровень глюкозы')
plt.show()

In [ ]:
X2 = df_task2[['avg_glucose_level']]
y2 = df_task2['stroke']

X_train, X_test, y_train, y_test = train_test_split(
    X2, y2, test_size=0.25, random_state=42, stratify=y2
)

print(f"Размер обучающей выборки: {X_train.shape[0]}")
print(f"Размер тестовой выборки: {X_test.shape[0]}")
print(f"Баланс классов в train: {np.bincount(y_train)}")

In [ ]:
log_model = LogisticRegression(max_iter=1000, random_state=42)
log_model.fit(X_train, y_train)

print(f"Коэффициент (наклон): {log_model.coef_[0][0]:.4f}")
print(f"Свободный член: {log_model.intercept_[0]:.4f}")

In [ ]:
y_pred = log_model.predict(X_test)
y_pred_proba = log_model.predict_proba(X_test)[:, 1]

print("Первые 10 предсказаний:", y_pred[:10])
print("Первые 10 вероятностей инсульта:", np.round(y_pred_proba[:10], 3))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Нет инсульта', 'Инсульт'],
            yticklabels=['Нет инсульта', 'Инсульт'])
plt.title('Матрица ошибок')
plt.ylabel('Истинные метки')
plt.xlabel('Предсказанные метки')
plt.show()

tn, fp, fn, tp = cm.ravel()
print(f"\nTP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")
print(f"Точность (accuracy): {(tp + tn) / (tp + tn + fp + fn):.3f}")

In [ ]:
print("Выводы по задаче 2:")
print("- Повышенный уровень глюкозы ассоциирован с более высоким риском инсульта (см. boxplot).")
print("- Модель логистической регрессии обучена на одном признаке — avg_glucose_level.")
print("- Несмотря на простоту, модель показывает определённую предсказательную способность.")
print("- Однако, из-за сильного дисбаланса классов (инсульт редок), точность может вводить в заблуждение.")
print("- Рекомендуется использовать другие метрики (например, F1-score, AUC-ROC) и больше признаков.")